In [1]:
import numpy as np

# difference = remove_items(list_a, items_to_remove)
def remove_items_slow(list_a, items_to_remove):
    if (len(list_a) == 0):
        return []
    first = list_a[0]
    rest = list_a[1:]
    next_result = remove_items_slow(rest, items_to_remove)
    if (first in items_to_remove):
        return next_result
    else:
        return [first] + next_result


def remove_items_iter(list_a, items_to_remove, result):
    if (len(list_a) == 0):
        return result
    first = list_a[0]
    rest = list_a[1:]
    if (first in items_to_remove):
        return remove_items_iter(rest, items_to_remove, result)
    else:
        return remove_items_iter(rest, items_to_remove, result + [first])


# difference = remove_items(list_a, items_to_remove)
def remove_items(list_a, items_to_remove):
    return remove_items_iter(list_a, items_to_remove, [])



# remapped_labels = remap_class_labels(classes)
def remap_class_labels(labels, classes):
    max_class = np.max(classes)
    new_indices = np.zeros((max_class+1))
    for i in range(0, max_class+1):
        if (i in classes):
            new_indices[i] = classes.index(i)

    remapped_labels = np.zeros((len(labels)))
    for i in range(0, len(labels)):
        label = labels[i]
        remapped_labels[i] = new_indices[label]

    return remapped_labels


#(selected_inputs, selected_labels) = select_classes(inputs, labels, classes)
# inputs are expected to be of shape (num_objects, ...)
def select_classes(inputs, labels, classes):
    number = inputs.shape[0]
    indices = np.zeros((number), dtype='bool')
    for c in classes:
        c_indices = (labels == c)
        indices = np.logical_or(indices, c_indices)

    selected_inputs = inputs[indices]
    selected_labels = labels[indices]
    remapped_labels = remap_class_labels(selected_labels, classes)
    return (selected_inputs, remapped_labels)

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [3]:
dataset = keras.datasets.mnist
#dataset = keras.datasets.fashion_mnist

# the data, split between train and test sets
(train_inputs, train_labels), (test_inputs, test_labels) = dataset.load_data()
number_of_classes = np.max([np.max(train_labels), np.max(test_labels)]) + 1
input_shape = train_inputs[0].shape

# Scale images to the [0, 1] range
train_inputs = train_inputs.astype("float32") / 255
test_inputs = test_inputs.astype("float32") / 255

# Right now, train_inputs[i] (for any i) is a 28x28 2D array.
# The Conv2D layer requires each input to be a 3D array.
# We use the np.expand_dims function to convert each training input to
# a 3D array of shape (28, 28, 1), to make it compatible with the Conv2D layer.
train_inputs = np.expand_dims(train_inputs, -1)
test_inputs = np.expand_dims(test_inputs, -1)

print("x_train shape:", train_inputs.shape)
print(train_inputs.shape[0], "train samples")
print(test_inputs.shape[0], "test samples")

11490434/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
#%% Create "big" dataset and "small" dataset

original_train_inputs = train_inputs
original_train_labels = train_labels
original_test_inputs = test_inputs
original_test_labels = test_labels

# compile list of classes. For the MNIST dataset, this will give us a
# list of 10 class labels, from 0 to 9.
original_classes = list(np.union1d(np.unique(train_labels), np.unique(test_labels)))

# We will exclude all samples of "small classes" from the big training dataset.
#small_classes = [2,3]
small_classes = [1,3,5,7,9]
big_classes = remove_items(original_classes, small_classes)

(big_train_inputs, big_train_labels) = select_classes(original_train_inputs,
                                                      original_train_labels,
                                                      big_classes)

(big_test_inputs, big_test_labels) = select_classes(original_test_inputs,
                                                    original_test_labels,
                                                    big_classes)

(small_train_inputs, small_train_labels) = select_classes(original_train_inputs,
                                                          original_train_labels,
                                                          small_classes)

(small_test_inputs, small_test_labels) = select_classes(original_test_inputs,
                                                        original_test_labels,
                                                        small_classes)

#%%

temp = small_train_inputs
(num, _, _, _) = temp.shape
temp = np.repeat(temp, 3, axis=3)
small_train_inputs = np.zeros((num,32,32,3))
small_train_inputs[:,2:30,2:30,:] = temp

temp = small_test_inputs
(num, _, _, _) = temp.shape
temp = np.repeat(temp, 3, axis=3)
small_test_inputs = np.zeros((num,32,32,3))
small_test_inputs[:,2:30,2:30,:] = temp

#%%

input_shape = small_train_inputs[0].shape
vgg16 = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=input_shape)

small_num_classes = len(small_classes)

# This is where we use the Sequential API to create the model.
# Notice that we create a list of layers, where we use all layers of the
# model except for the last one, and we add a new fully connected
# output layer.
refined_model = keras.Sequential([keras.Input(shape=input_shape)]+
                                 vgg16.layers +
                                 [layers.Flatten(),
                                  layers.Dropout(0.5),
                                  layers.Dense(512, activation="tanh"),
                                  layers.Dropout(0.5),
                                  layers.Dense(small_num_classes, activation="softmax")])

# We freeze the weights of all layers except the ones in the (new) output layer.
for i in range(0, len(vgg16.layers)):
    refined_model.layers[i].trainable = False


refined_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer="adam", metrics=["accuracy"])
refined_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        multiple                  0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                         

In [5]:
# train the model (essentially, train the weights of the last layer) on the
# "small" dataset.
train_size = 100  # this specifies the size of the "small" training set
epochs = 100
batch_size = 4
hist_1dense = refined_model.fit(small_train_inputs[0:train_size],
                  small_train_labels[0:train_size],
                  epochs=epochs, batch_size=batch_size,
#                  validation_data = (small_test_inputs, small_test_labels)
                  )

Epoch 1/100
25/25 [==============================] - 2s 57ms/step - loss: 1.8750 - accuracy: 0.3700
Epoch 2/100
25/25 [==============================] - 1s 56ms/step - loss: 1.1475 - accuracy: 0.5700
Epoch 3/100
25/25 [==============================] - 1s 57ms/step - loss: 0.8822 - accuracy: 0.6900
Epoch 4/100
25/25 [==============================] - 1s 56ms/step - loss: 0.8162 - accuracy: 0.7200
Epoch 5/100
25/25 [==============================] - 1s 57ms/step - loss: 0.7198 - accuracy: 0.7300
Epoch 6/100
25/25 [==============================] - 2s 82ms/step - loss: 0.6423 - accuracy: 0.7300
Epoch 7/100
25/25 [==============================] - 2s 78ms/step - loss: 0.5976 - accuracy: 0.8300
Epoch 8/100
25/25 [==============================] - 1s 55ms/step - loss: 0.6295 - accuracy: 0.7800
Epoch 9/100
25/25 [==============================] - 1s 56ms/step - loss: 0.5458 - accuracy: 0.8200
Epoch 10/100
25/25 [==============================] - 1s 56ms/step - loss: 0.4311 - accuracy: 0.8100

In [6]:
test_loss, test_acc = refined_model.evaluate(small_test_inputs, small_test_labels, verbose=2)
print('\nTest accuracy: %.2f' % (test_acc * 100))
#refined_model.save('fashion_mnist_refined5_tsize20_epochs100.h5')

159/159 - 54s - loss: 0.4986 - accuracy: 0.8504 - 54s/epoch - 337ms/step

Test accuracy: 85.04
